In [1]:
## Dependencies
import sys, os
import pandas as pd
import re
import glob
import numpy as np
from pprint import pprint

## Scikitlearn
import sklearn as sklearn
from tika import parser # pip install tika
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer


## Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import models
#from gensim.models.coherencemodel import CoherenceModel
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

## NLTK
import nltk as nltk
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

## Spacy
import spacy
import itertools as it
# nlp = spacy.load('en')
nlp = spacy.load(r'C:\Users\keg827\AppData\Local\Continuum\anaconda3\Lib\site-packages\en_core_web_sm\en_core_web_sm-2.3.1')

## Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from wordcloud import WordCloud
# load and show an image with Pillow
from PIL import Image


C:\Users\keg827\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\backend_bases.py:55: DeprecationWarning: PILLOW_VERSION is deprecated and will be removed in a future release. Use __version__ instead.
  from PIL import PILLOW_VERSION
C:\Users\keg827\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\backend_bases.py:55: DeprecationWarning: PILLOW_VERSION is deprecated and will be removed in a future release. Use __version__ instead.
  from PIL import PILLOW_VERSION


In [2]:
print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))
#print('The pillow version is {}.'.format(PILLOW_VERSION))

The nltk version is 3.5.
The scikit-learn version is 0.23.1.


In [3]:
directory = "News_Industry"
files = list(glob.glob(os.path.join(directory,'*.*')))
print(files)
#https://stackoverflow.com/questions/34000914/how-to-create-a-list-from-filenames-in-a-user-specified-directory-in-python
#https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
#https://stackoverflow.com/questions/33912773/python-read-txt-files-into-a-dataframe

['News_Industry\\Bibliography.10AGGRESSION AND PHYSICAL HEALTH IN MARRIED WOMEN.pdf', 'News_Industry\\Bibliography.12Impact of Socio-demographic Factors on Awareness of Smoking Effects on Oral Health among Smokers and.pdf', 'News_Industry\\Bibliography.17Health-Promoting Factors related to lifestyle among nursing students in University of Hail.pdf', 'News_Industry\\Bibliography.17Multinomial logit analysis of the effects of five different app-based incentives to encourage cyclin.pdf', 'News_Industry\\Bibliography.1PREVALENCE OF DYSLIPIDEMIA IN YOUNG ADULTS.pdf', 'News_Industry\\Bibliography.20Risk Factors for Atherosclerotic Cardiovascular Disease in the South Asian Population.pdf', 'News_Industry\\Bibliography.29Is the Gay Community the Neo-marginalised of Modern Society_.pdf', 'News_Industry\\Bibliography.33A Biological Effect of Sex Hormone Binding Globulin and Testosterone in Polycystic Ovary Syndrome (P.pdf', 'News_Industry\\Bibliography.34DETERMINANTS OF DEPRESSION ANXIETY STRESS

In [4]:
#https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file
document_list = []
for f in files:
    raw = parser.from_file(f)
    document_list.append(raw)

In [5]:
text_df = pd.DataFrame(document_list)
text_df.head()
#print(text_df["content"][1])

,metadata,content,status
0,"{'Content-Type': 'application/pdf', 'Creation-...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,200
1,"{'Content-Type': 'application/pdf', 'Creation-...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,200
2,"{'Content-Type': 'application/pdf', 'Creation-...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,200
3,"{'Content-Type': 'application/pdf', 'Creation-...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,200
4,"{'Content-Type': 'application/pdf', 'Creation-...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,200


In [6]:
## Create a new column with the title from the metadata key called dc:title

text_df['title'] = [value.get('dc:title') for value in text_df["metadata"]]

text_df.head()
text_df.keys()

## https://stackoverflow.com/questions/44218812/pandas-add-columns-to-a-dataframe-based-in-dict-from-one-of-the-columns

Index(['metadata', 'content', 'status', 'title'], dtype='object')

In [7]:
## Pre-process the text to lower case, remove special characters, etc. 
## https://kavita-ganesan.com/extracting-keywords-from-text-tfidf/#.X7RHltBKiUn
## Test regex here: https://pythex.org/

def preprocess(text):
    
    ## Lowercase words
    text_lower = text.lower()
    
    ## Remove Emails from text
    ## if you need to match a \, you can precede them with a backslash to remove their special meaning: \\.
    ## \S matches any non-whitespace character; this is equivalent to the class [^ \t\n\r\f\v].
    ## \s Matches any whitespace character; this is equivalent to the class [ \t\n\r\f\v]
    ## Code below matches any character, then an @ sign, then more characters, end matching when a white space is found.
    text_email = re.sub('\\S*@\\S*\\s?', '', text_lower) 
    
    ## Remove URLS from text
    ## https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python/40823105#40823105
    ## text_urls = re.sub(r'http\S+', '', text_email)
    ## https://www.geeksforgeeks.org/python-check-url-string/
    text_urls = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",'', text_email)
    
    
    ## Remove tabs and new lines from text
    ## https://stackoverflow.com/questions/16355732/how-to-remove-tabs-and-newlines-with-a-regex
    ## \s Matches any whitespace character; this is equivalent to the class [ \t\n\r\f\v]
    text_spaces = re.sub(r'\s+',' ',text_urls)
        
    ## Remove \n from text
    text_space_character = text_spaces.replace('\n','')
    
    ## Remove \t from text
    text_tab_character = text_space_character.replace('\t','')
    
    ## Remove special characters and numbers
    ## \W matches any non-alphanumeric character; this is equivalent to the class [^a-zA-Z0-9_], such as space, percent sign, underscore, pipe, colon, semicolon, 
    ## \d matches any decimal digit; this is equivalent to the class [0-9]
    #text_numbers = re.sub("(\\d|\\W)+"," ",text_tab_character)
    
    ## Remove tags
    ##text_tags = re.sub("","",text_numbers)
    
    ## Remove special characters and space, but leave in periods and numbers
    ## ^ means any character except. So [^5] will match any character except '5'
    ## [^a-zA-Z0-9_] matches any non-alphanumeric character.
    text_special = re.sub('[^A-Za-z0-9.]+|\s',' ',text_tab_character)
    
    ## Remove numbers with decimals
    ## https://stackoverflow.com/questions/39582859/python-regex-remove-numbers-and-numbers-with-punctaution
    ## will match an integer or decimal number followed by 1 or more spaces
    text_decimals = re.sub(r'\b\d+(?:\.\d+)?\s+', '', text_special)
    
    ## Remove numbers from text
    text_allnums = re.sub("\d+", "", text_decimals)
    
    ## Remove a sepcial list of terms
    ## https://stackoverflow.com/questions/15435726/remove-all-occurrences-of-words-in-a-string-from-a-python-list
    
    REMOVE_LIST = ['right reserved section',
                   'reserved section',
                   'all rights',
                   "length word byline", 
                   'length words',
                   "byline", 
                   "word byline",
                   "journal code", 
                   "vol.",
                   "no.",
                   "keywords",
                   "dr", 
                   "publication type magazine",
                   "type magazine",
                   "magazine",
                   "type newspaper",
                   "publication type newspaper",
                   'newspaper',
                   "group right reserved",
                   'section:',
                   'copyright',
                   'body',
                   'length:',
                   'keywords:',
                   'introduction',
                   'page',
                   'methodology',
                   'table',
                   'discussion',
                   'conclusions',
                   'references',
                   'classification',
                   'language',
                   'industry',
                   'geographic',
                   'load-date',
                   'end of document',
                   'mg dl',
                   'mg'
                   
                  ]

    remove = '|'.join(REMOVE_LIST)
    regex = re.compile(r'\b('+remove+r')\b', flags=re.IGNORECASE)
    text_special_remove = regex.sub("", text_allnums)

    return text_special_remove

## New column "preprocess" is formed from applying pre_process function to each item in the "content" column in dataframe
text_df['preprocess'] = text_df['content'].apply(lambda x:preprocess(x))

print(text_df['preprocess'][1])
print("-----------------------------------------------------")
print("This is the title: ",text_df['title'][1])

#https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

<>:46: DeprecationWarning: invalid escape sequence \s
<>:54: DeprecationWarning: invalid escape sequence \d
<>:46: DeprecationWarning: invalid escape sequence \s
<>:54: DeprecationWarning: invalid escape sequence \d
<>:46: DeprecationWarning: invalid escape sequence \s
<>:54: DeprecationWarning: invalid escape sequence \d
<ipython-input-7-5250be00e385>:46: DeprecationWarning: invalid escape sequence \s
  text_special = re.sub('[^A-Za-z0-9.]+|\s',' ',text_tab_character)
<ipython-input-7-5250be00e385>:54: DeprecationWarning: invalid escape sequence \d
  text_allnums = re.sub("\d+", "", text_decimals)


 impact of socio demographic factors on awareness of smoking effects on oral health among smokers and  smokers dental patients visiting private clinics impact of socio demographic factors on awareness of smoking effects on oral health among smokers and  smokers dental patients visiting private clinics biomedica september monday  biomedical research society   vol. no.   asad r. nadeem m. christopher m. ahmed h. awais f. and majeed h.a.   awareness smokers  smokers oral health.  the global tobacco epidemic is one of the serious and major public health concerns. it causes more than eight million people deaths around the world. of these fatalities seven million deaths are occurring due to direct consumption of tobacco while million mortalities of  smokers are the result of passive smoke.owing to tobacco consumption low and middle income countries are the most vulnerable countries contributing to the highest morbidity and mortality rates where currently billion i.e. of the world s smokers r

In [8]:
## Sentence detection and segmentation with Spacy
## https://github.com/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb

def sentence_detection(text):
    
    parsed_text = nlp(text)
#     print(type(parsed_text))
#     for num, sentence in enumerate(parsed_text.sents):
#         print('Sentence {}:'.format(num + 1))
#         print(sentence)
#         print('')
        
#     for num, entity in enumerate(parsed_text.sents):
#         print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
#         print('')
    lemma = []
    # Extract the lemma for each token and join
    for num, sentence in enumerate(parsed_text.sents):
        print(sentence)
        
        lemma = " ".join([token.lemma_ for token in sentence])
   
    return lemma

## New column "parsed" is formed from applying sentence detection function to each item in the "content" column in dataframe
text_df['parsed'] = text_df['preprocess'].apply(lambda x:sentence_detection(x))

print(text_df['parsed'][1])
print("-----------------------------------------------------")
print("This is the title: ",text_df['title'][1])

 aggression and physical health in married women aggression and physical health in married women journal of postgraduate medical institute december tuesday  postgraduate medical institute   vol.
no.   
faiza shafique and riffat sadiq   aggression health women  
aggression is an instinctive drive of a person and a dark side of human nature.
it includes a variety of range of behaviors.
aggression involves verbal and physical assault therefore its expression results in intense violence towards others.
aggression is an unwanted and maladaptive behavior causing damage and obliteration.
it is exhibited in different forms encompassing physical aggression verbal aggression anger and hostility.
a person with physical aggression causing physical and emotional harm others while harming or hurting someone verbally is described as verbal aggression.
anger involves physiological activation determining emotional and affective state.
hostility includes feelings of cynicism mistrust and opposition.
wit

 impact of socio demographic factors on awareness of smoking effects on oral health among smokers and  smokers dental patients visiting private clinics impact of socio demographic factors on awareness of smoking effects on oral health among smokers and  smokers dental patients visiting private clinics biomedica september monday  biomedical research society   vol.
no.   
asad r. nadeem m. christopher m. ahmed h. awais f. and majeed h.a.   
awareness smokers  smokers oral health.  
the global tobacco epidemic is one of the serious and major public health concerns.
it causes more than eight million people deaths around the world.
of these fatalities seven million deaths are occurring due to direct consumption of tobacco while million mortalities of  smokers are the result of passive smoke.owing to tobacco consumption low and middle income countries are the most vulnerable countries contributing to the highest morbidity and mortality rates
where currently billion i.e. of the world s smoker

 health promoting factors related to lifestyle among nursing students in university of hail health promoting factors related to lifestyle among nursing students in university of hail asian journal of nursing education and research december  a v publications distributed by contify.com  reserved   .
v. indra  .
v. indra
assistant professor university of hail kingdom of saudi arabia corresponding author email abstract health promotion is an important determinant of individual health status which held the individual responsible for his own health.
health promotion behaviors are directed toward achieving a higher level of wellness personal fulfillment and self actualization.
health related behaviors in young age are important factor that affects the individual risk for  communicable diseases and other disorders later in life.
it is evident that promoting healthy lifestyle behaviors among university students is essential to decrease disease risk later in adulthood thus it is important to inv

 multinomial logit analysis of the effects of five different app based incentives to encourage cycling to work multinomial logit analysis of the effects of five different app based incentives to encourage cycling to work iet intelligent transport systems december  
institution of electrical engineers   
pg.
vol.
no.
x   bingyuanhuang tiagofioreze tomthomas ericvan berkum   and
background road transport contributes about one fifth of the eu s total emissions of carbon dioxide according to the european commission .
another worrying societal issue is that the population in every eu country will be more obese by unless europe s governments take immediate action according to new projections released by the world health organisation .
active forms of transport especially cycling can provide part of the solution for both challenges.
cycling is an environmentally friendly mode of transport and a very effective way to increase physical activity.
several studies have shown that cycling has a pro

 prevalence of dyslipidemia in young adults.
prevalence of dyslipidemia in young adults.
the professional medical journal may sunday  
the professional medical journal   vol.
no.   
mir tahir hussain talpur muhammad tauqeer katbar khalil ullah
shabir kashif ullah shabir uzair yaqoob shah jabeen and danish
zia  
key words dyslipidemia hyperlipidemia ldl hdl triglyceride young adults.  
the term dyslipidemia signifies abnormal lipid or fat content in the blood including triglycerides phospholipids cholesterol lipoprotein and other fats.
lipoproteinemia denotes fatty substances that circulate in the blood attached with proteins and when they exceed the normal level it is called hyperlipoproteinemia.similarly when cholesterol or triglycerides are increased the term given to this condition is hypercholesterolemia and hypertriglyceridemia respectively.  
optimal levels of low density lipoprotein ldl and high density lipoprotein hdl cholesterol in young adults are independently associated wit

...

 clinical trial cardia salt sensitivity of blood pressure ssbp clinical trial cardia salt sensitivity of
blood pressure ssbp health daily digest february tuesday  ht media ltd.  
reserved  dateline u.s.  
u.s. feb.
clinicaltrials.gov registry received information related to the study nct titled cardia salt sensitivity of blood pressure ssbp on feb. .
brief summary salt sensitivity of blood pressure ssbp is defined as the change in blood pressure bp in relation to change in salt intake.
an increase in bp from low to high salt diet is common and associated with an increased risk of cardiovascular morbidity and mortality even among normotensive individuals.
yet the pathophysiology of ssbp is  well understood.
the prevailing paradigm is that abnormalities of neurohormones that regulate sodium na retention and excretion and or na transporting pathways create na imbalances that underlie susceptibility to ssbp.
as a homeostatic mechanism bp fluctuates to maintain na balance i.e. higher bp is 

 coronary stents as effective as cabg in diabetes patients coronary stents as effective as cabg in diabetes patients internal medicine news october  elsevier inc.
international medical news group   
pg.
vol.
no.
issn   
mitchel l. zoler philadelphia bureau  munich patients with diabetes who received coronary stents fared just as well as similar patients who underwent coronary bypass surgery in a randomized study with patients with year of follow up.
the results seemed to disprove the conventional wisdom that percutaneous coronary intervention pci is  a good option for patients with diabetes because of their greater risk of restenosis compared with nondiabetic patients .
akhil kapur said at the annual congress of the european society of cardiology.
but some experts were skeptical of the finding saying that a study with a total of patients wasn t large enough to definitively address the issue.
five hundred patients is small for any comparison of pci and coronary surgery in patients with 

the patients average age was years and about were on insulin treatment.
the study received support from cordis the company that markets cypher stents as well as many other device and drug companies.
the study s primary sponsor was hammersmith hospitals nhs trust london.
.
kapur said he had relevant disclosures.
the study s primary end point
was the combined rate of death nonfatal myocardial infarction or nonfatal stroke after year.
the rate was in patients treated with cabg and in the patients treated with  of pci effective as surgery in diabetes in small trial pci a difference that was  statistically significant reported .
kapur a cardiologist at the london chest hospital.
as in the other major comparison of pci and cabg presented at the meeting the syntax study the rate of stroke was significantly lower in patients treated with pci than in patients treated with cabg .
on the other hand the pci patients had a higher rate of nonfatal mis although  significantly higher than the cabg pat

 even secondhand smoke raises diabetes risk
even secondhand smoke raises diabetes risk clinical endocrinology news
may  elsevier inc.
international medical news group   
pg.
vol.
no.
issn   
melinda tanzola contributing writer  both smoking and exposure to secondhand smoke increase the risk of developing glucose intolerance a prospective cohort study indicates.
over years the development of glucose intolerance was highest among smokers followed by people who had never smoked but had secondhand smoke exposure previous smokers and those who neither smoked  had secondhand smoke exposure .
this is the first study to show that secondhand smoke is independently associated with a risk of developing glucose intolerance the researchers said bmj april epub doi.bmj.. .
in the coronary artery risk development in young adults cardia study .
thomas k. houston of the birmingham ala.
veterans affairs medical center and his associates enrolled young adults aged from four u.s. cities.
the cohort include

 stable weight averts risk factors in young adults stable weight averts risk factors in young adults internal medicine news february  
elsevier inc.
international medical news group   
pg.
vol.
no.
issn   
bruce jancin denver bureau  new orleans young adults who maintain a stable  weight into middle age or who at least manage to gain less than the american average of about pound per year may largely prevent progression of other cardiovascular risk factors and development of metabolic syndrome donald m. lloyd jones m.d.
said at the annual scientific sessions of the american heart association.
this new observation from the national heart lung and blood institute sponsored coronary artery risk development in young adults cardia study carries a hopeful message that has important implications.
it suggests that greater public health emphasis should be placed on achieving weight stabilization in young and middle aged adults.
that s a more realistic goal for most people than long term maintena

In [ ]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(text):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    for review in text:
        yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(text):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(text),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

## New column "parsed" is formed from applying sentence detection function to each item in the "content" column in dataframe
text_df['lemmatized'] = text_df['preprocess'].apply(lambda x:lemmatized_sentence_corpus(x))

print(text_df['lemmatize'][1])

In [ ]:
def unigram_sentences(text):
    unigram_sentences = LineSentence(text)
    for unigram_sentence in it.islice(unigram_sentences, 230, 240):
        print(u' '.join(unigram_sentence))
        print(u'')

## New column "parsed" is formed from applying sentence detection function to each item in the "content" column in dataframe
text_df['unigram'] = text_df['lemmatize'].apply(lambda x:unigram_sentences(x))

In [ ]:
def lemmatize(text):

    for sent in text:
        print(sent)
        yield u' '.join([token.lemma_ for token in sent])

## New column "lemmatize" is formed from applying pre_process function to each item in the "preprocess" column in dataframe
text_df['lemmatize'] = text_df['parsed'].apply(lambda x:lemmatize(x))

print(text_df['lemmatize'][1])

In [ ]:
## Lemmatize documents

def lemmatize(text):
    #print(text)
    
    ## Initiate a lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    ## Tokenize: Split the sentence into words
    word_list = nltk.word_tokenize(text)
    print(word_list)
    
    ## Lemmatize list of words and join
    lemmatized_text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
                
    return lemmatized_text

## New column "lemmatize" is formed from applying pre_process function to each item in the "preprocess" column in dataframe
text_df['lemmatize'] = text_df['parsed'].apply(lambda x:lemmatize(x))

print(text_df['lemmatize'][1])
